In [1]:
# EXECUTE FIRST

# computational imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, KNNBasic, NormalPredictor,BaselineOnly,KNNWithMeans,KNNBaseline
from surprise import SVD, SVDpp, NMF, SlopeOne, CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import accuracy

import random
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
matplotlib.style.use('ggplot')
# for reading files from urls
import urllib.request
# display imports
from IPython.display import display, IFrame
from IPython.core.display import HTML

# import notebook styling for tables and width etc.
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

<font size=18>Lesson 14 - Self-Assessment Solutions</font>

# *Self-Assessment: Setting up the File*

In [2]:
# load the data
import pandas as pd
import numpy as np
bx = pd.read_csv('./data/BX-Book-Ratings-3000.csv')
bx.head()

,User-ID,ISBN,Book-Rating
0,6251,345370775,1
1,6251,044021145X,1
2,6251,312983271,1
3,6251,080410526X,1
4,6251,743418174,1


In [3]:
print("Mean book rating:     ", '%.2f' % bx['Book-Rating'].mean())

Mean book rating:      2.63


In [4]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = bx.copy()
y = bx['User-ID']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify=y, random_state=42)

# *Self-Assessment: Baseline RMSE to Assess Model Performance*

In [5]:
#verify the median of the data
print(f"The median of this rating range is {np.median(np.arange(np.min(bx['Book-Rating']), (np.max(bx['Book-Rating']) + 1)))}")

#Define the baseline model to always the scale median.
def baseline(user_id, item_id, scale_median,  *args):
    return scale_median

The median of this rating range is 6.0


In [6]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model, X_test, *args):
    
    #Construct a list of user-book tuples from the testing dataset
    id_pairs = zip(X_test[X_test.columns[0]], X_test[X_test.columns[1]])
    
    #Predict the rating for every user-item tuple
    y_pred = np.array([cf_model(user, item, *args) for (user, item) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test[X_test.columns[2]])
    
    #Return the final RMSE score
    return mean_squared_error(y_true, y_pred, squared=False)

In [7]:
score(baseline, X_test, 6)

4.703780985075257

# *Self-Assessment: Weighted Mean User-Based Filter*

In [8]:
#Build the ratings matrix using pivot_table function
#r_matrix = X_train.pivot_table(values='Book-Rating', index='User-ID', columns='ISBN')
r_matrix = X_train.pivot(values='Book-Rating', index='User-ID', columns='ISBN')

r_matrix.head()

ISBN,006101351X,014025448X,014028009X,034540288X,038079487X,043935806X,044021145X,044022165X,044023722X,044651652X,...,743418174,767902521,767905180,786868716,786881852,804106304,804114986,805063897,842329129,971880107
User-ID,,,,,,,,,,,,,,,,,,,,,
6251,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7346,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,1.0,NaN,NaN,NaN
11601,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11676,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [10]:
r_matrix_dummy.head()

ISBN,006101351X,014025448X,014028009X,034540288X,038079487X,043935806X,044021145X,044022165X,044023722X,044651652X,...,743418174,767902521,767905180,786868716,786881852,804106304,804114986,805063897,842329129,971880107
User-ID,,,,,,,,,,,,,,,,,,,,,
6251,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7346,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0
11601,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11676,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [12]:
#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head()

User-ID,6251,6575,7346,11601,11676,13552,14521,16795,21014,23768,...,238781,254465,258534,260897,261829,265115,265313,266226,269566,274308
User-ID,,,,,,,,,,,,,,,,,,,,,
6251,1.000000,0.019892,0.030961,0.005078,0.142988,0.048059,0.064752,0.018610,0.002779,0.035858,...,0.015235,0.023108,0.006664,0.026021,0.018473,0.029876,0.003219,0.052636,0.079578,0.045950
6575,0.019892,1.000000,0.001540,0.022224,0.115155,0.002390,0.000000,0.001018,0.132236,0.046586,...,0.075006,0.145943,0.014581,0.001582,0.000000,0.012711,0.019368,0.014397,0.051694,0.000000
7346,0.030961,0.001540,1.000000,0.003931,0.193686,0.016909,0.019491,0.024488,0.002151,0.104082,...,0.043239,0.030895,0.000000,0.044759,0.017873,0.256943,0.007474,0.112041,0.281033,0.032603
11601,0.005078,0.022224,0.003931,1.000000,0.002773,0.030508,0.005024,0.031187,0.003880,0.006260,...,0.007092,0.002934,0.006204,0.407819,0.029023,0.046359,0.000000,0.055132,0.020838,0.085563
11676,0.142988,0.115155,0.193686,0.002773,1.000000,0.009544,0.212180,0.081307,0.022762,0.088125,...,0.033282,0.022946,0.000000,0.018948,0.001261,0.010877,0.275982,0.079053,0.027163,0.043917


In [13]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_wmean(user_id, item_id, ratings_matrix, c_sim_matrix, median_rating):
    
    #Check if item_id exists in r_matrix
    if item_id in ratings_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = c_sim_matrix[user_id]
        
        #Get the user ratings for the item in question
        i_ratings = ratings_matrix[item_id]
        
        #Extract the indices containing NaN in the i_ratings series
        idx = i_ratings[i_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        i_ratings = i_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)

        #Compute the final weighted mean
        if sim_scores.sum()>0:
            wmean_rating = np.dot(sim_scores, i_ratings)/ sim_scores.sum()
        else:  # user had zero cosine similarity with other users
            wmean_rating = median_rating

    else:
        #Default to the median in the absence of any information
        wmean_rating = median_rating
    
    return wmean_rating

In [14]:
score(cf_wmean, X_test, r_matrix, cosine_sim, 6)

3.607093266358255

The RMSE with the user-based collaborative filter is 3.61 compared to 4.70 for the baseline model, so predicted ratings are more precise.  

# *Self-Assessment: Weighted Mean Item-Based Filter - Solution*

In [15]:
#Build the ratings matrix using pivot_table function
#r_matrix = X_train.pivot_table(values='Book-Rating', index='ISBN', columns='User-ID')
r_matrix_item = X_train.pivot(values='Book-Rating', index='ISBN', columns='User-ID')

r_matrix_item.head()

User-ID,6251,6575,7346,11601,11676,13552,14521,16795,21014,23768,...,238781,254465,258534,260897,261829,265115,265313,266226,269566,274308
ISBN,,,,,,,,,,,,,,,,,,,,,
006101351X,NaN,NaN,1.0,NaN,9.0,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
014025448X,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN
014028009X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
034540288X,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
038079487X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [16]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_item_dummy = r_matrix_item.copy().fillna(0)

In [17]:
# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim_item = cosine_similarity(r_matrix_item_dummy, r_matrix_item_dummy)

In [18]:
#Convert into pandas dataframe 
cosine_sim_item = pd.DataFrame(cosine_sim_item, index=r_matrix_item.index, columns=r_matrix_item.index)

cosine_sim_item.head(10)

ISBN,006101351X,014025448X,014028009X,034540288X,038079487X,043935806X,044021145X,044022165X,044023722X,044651652X,...,743418174,767902521,767905180,786868716,786881852,804106304,804114986,805063897,842329129,971880107
ISBN,,,,,,,,,,,,,,,,,,,,,
006101351X,1.000000,0.005847,0.044958,0.035806,0.000000,0.051717,0.036379,0.032858,0.044481,0.064775,...,0.032187,0.047741,0.009323,0.007262,0.006989,0.058881,0.043183,0.029235,0.006989,0.026854
014025448X,0.005847,1.000000,0.009320,0.000000,0.044639,0.000000,0.031109,0.000000,0.007245,0.000000,...,0.033029,0.049897,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071714,0.010206
014028009X,0.044958,0.009320,1.000000,0.000000,0.000000,0.000000,0.032617,0.000000,0.000000,0.034416,...,0.030783,0.006341,0.000000,0.000000,0.000000,0.391058,0.189290,0.279600,0.000000,0.007134
034540288X,0.035806,0.000000,0.000000,1.000000,0.000000,0.032827,0.063500,0.000000,0.000000,0.150756,...,0.044947,0.000000,0.000000,0.025351,0.000000,0.000000,0.000000,0.000000,0.048795,0.000000
038079487X,0.000000,0.044639,0.000000,0.000000,1.000000,0.000000,0.000000,0.055744,0.177880,0.146524,...,0.051876,0.033748,0.063267,0.024639,0.000000,0.049947,0.000000,0.000000,0.539464,0.000000
043935806X,0.051717,0.000000,0.000000,0.032827,0.000000,1.000000,0.041690,0.005021,0.034955,0.197952,...,0.014754,0.328266,0.000000,0.003329,0.032035,0.000000,0.000000,0.000000,0.006407,0.131306
044021145X,0.036379,0.031109,0.032617,0.063500,0.000000,0.041690,1.000000,0.019424,0.005635,0.191460,...,0.031395,0.038806,0.066136,0.003220,0.006197,0.000000,0.191460,0.000000,0.037182,0.007938
044022165X,0.032858,0.000000,0.000000,0.000000,0.055744,0.005021,0.019424,1.000000,0.020357,0.023057,...,0.175295,0.000000,0.039823,0.031018,0.074629,0.010480,0.046114,0.000000,0.000000,0.114708
044023722X,0.044481,0.007245,0.000000,0.000000,0.177880,0.034955,0.005635,0.020357,1.000000,0.107019,...,0.087744,0.049298,0.057762,0.076484,0.017319,0.012160,0.053510,0.000000,0.000000,0.033276


In [19]:
#Item-Based Collaborative Filter using Weighted Mean Ratings
def cf_item_wmean(user_id, item_id, ratings_matrix, c_sim_matrix, median_rating):
    
    #Check if user exists in r_matrix
    if user_id in ratings_matrix:
        
        #Get the similarity scores for the item in question with every other item
        sim_scores = c_sim_matrix[item_id]
        
        #Get the user ratings for the book in question
        u_ratings = ratings_matrix[user_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = u_ratings[u_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        u_ratings = u_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        if sim_scores.sum() > 0:
            wmean_rating = np.dot(sim_scores, u_ratings)/ sim_scores.sum()
        else: # the book has zero cosine similarity with other books
            wmean_rating = median_rating
    
    else:
        #Default to a rating of 6.0 in the absence of any information
        wmean_rating = median_rating
    
    return wmean_rating

In [20]:
score(cf_item_wmean, X_test, r_matrix_item, cosine_sim_item, 6)

3.4119539180908327

The weighted-mean item-based collaborative filter is the best so far at RMSE = 3.41.  The weighted-mean item-based collaborative filter had RMSE = 3.61 and the baseline model had RMSE = 4.70.  

# *Self-Assessment: kNN-Based Collaborative Filter - Solution*

In [21]:
bx.head()

,User-ID,ISBN,Book-Rating
0,6251,345370775,1
1,6251,044021145X,1
2,6251,312983271,1
3,6251,080410526X,1
4,6251,743418174,1


In [22]:
#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader(rating_scale=(1,11))

#Create the dataset to be used for building the filter
#data = Dataset.load_from_df(ratings, reader)
data = Dataset.load_from_df(bx, reader)

#Define the algorithm object; in this case kNN
random.seed(1)
np.random.seed(1)
knn = KNNBasic(k=5, verbose=False)

#Evaluate the performance in terms of RMSE
from surprise.model_selection import cross_validate
knn_cv = cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=True)
#to extract the mean RMSE, we need to get the mean of the test_rmse values
knn_RMSE = np.mean(knn_cv['test_rmse'])
print(f'\nThe RMSE across five folds was {knn_RMSE}')

Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.6956  3.7194  3.8178  3.7849  3.4263  3.6888  0.1384  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    

The RMSE across five folds was 3.688801997695312


The RMSE for each model used so far are stated below ranked from best to worst:

- weighted-mean item-based collaborative filter: RMSE = 3.41 

- weighted-mean item-based collaborative filter: RMSE = 3.61 

- kNN-based collaborative filter: (average) RMSE = 3.69 (note that this one will vary slightly if you didn't set a seed or if you use a different seed)

- baseline model: RMSE = 4.70.  

# *Self-Assessment: kNNBasic Item-based Collaborative Filter - Solution*

In [23]:
#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader(rating_scale=(1,11))

#Create the dataset to be used for building the filter
#data = Dataset.load_from_df(ratings, reader)
data = Dataset.load_from_df(bx, reader)


sim_options = {'user_based': False  # compute  similarities between items
               }

#Define the algorithm object; in this case kNN
random.seed(1)
np.random.seed(1)
knn = KNNBasic(k=5, sim_options=sim_options)

#Evaluate the performance in terms of RMSE
from surprise.model_selection import cross_validate
knn_cv = cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=True)
#to extract the mean RMSE, we need to get the mean of the test_rmse values
knn_RMSE = np.mean(knn_cv['test_rmse'])
print(f'\nThe RMSE across five folds was {knn_RMSE}')

#re-train on the whole dataset
trainset = data.build_full_trainset()
knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.4648  3.4779  3.4987  3.4872  3.2932  3.4444  0.0764  
Fit time          0.01    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    

The RMSE across five folds was 3.444369689909382
Computing the msd similarity matrix...
Done computing similarity matrix.


The RMSE for each model used so far are stated below ranked from best to worst:


- weighted-mean item-based collaborative filter: RMSE = 3.41 

- kNN-based item-based collaborative filter: (average) RMSE = 3.44

- weighted-mean item-based collaborative filter: RMSE = 3.61 

- kNN-based user-based collaborative filter: (average) RMSE = 3.69 (note that this one will vary slightly if you didn't set a seed or if you use a different seed)

- baseline model: RMSE = 4.70.  

# *Self-Assessment: Hybrid Recommender*

In [24]:
# load the data
import pandas as pd
import numpy as np
bx = pd.read_csv('./data/BX-Book-Ratings-3000.csv')
bx.head(5)

,User-ID,ISBN,Book-Rating
0,6251,345370775,1
1,6251,044021145X,1
2,6251,312983271,1
3,6251,080410526X,1
4,6251,743418174,1


In [25]:
#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader(rating_scale=(1,11))

#Create the dataset to be used for building the filter
#data = Dataset.load_from_df(ratings, reader)
data = Dataset.load_from_df(bx, reader)

#train a knn item-based collaborative filter - don't set k, just let it pick
sim_options = {'user_based': False  # compute  similarities between items
               }

# Retrieve the trainset.
trainset = data.build_full_trainset()

#Define the algorithm object; in this case item-based kNNBasic
random.seed(1)
np.random.seed(1)
knn = KNNBasic(sim_options=sim_options)

#fit the data
knn.fit(trainset)


#Build the SVD based Collaborative filter
svd = SVD()
random.seed(1)
np.random.seed(1)

#fit the data
svd.fit(trainset)

#test a couple of predictions (Note that ISBN is a string)
print(knn.predict(31315, '446606189'))
print(svd.predict(31315, '446606189'))

Computing the msd similarity matrix...
Done computing similarity matrix.


user: 31315      item: 446606189  r_ui = None   est = 8.84   {'actual_k': 30, 'was_impossible': False}
user: 31315      item: 446606189  r_ui = None   est = 7.39   {'was_impossible': False}


In [26]:
#build the hybrid function
def hybrid(ratings, userid, item_algo, user_algo, item_weight, N):
    '''
    Parameters
    ratings: the ratings dataframe we're working with
    userid: the user for whom we are making predictions
    item_algo: the trained Surprise item-based collaborative filter
    user_algo: the trained Surprise user-based collaborative filter
    N: the number of predictions to return
    returns
    a dataframe of top recommendations
    '''

    #first get a dataframe of unique books
    sim_items = ratings.copy().drop(columns=['User-ID', 'Book-Rating']).drop_duplicates()
    #generate the predicted this user's predicted rating for each of them based on the item-based filter
    sim_items['iPrediction'] = sim_items.apply(lambda x: item_algo.predict(userid, x['ISBN']).est, axis=1)

    #add the predictions based on the user-based collaborative filter
    sim_items['uPrediction'] = sim_items.apply(lambda x: user_algo.predict(userid, x['ISBN']).est, axis=1)

    #weight the item-based collaborative filter by item_weight and the user-based collaborative filter by 1-item_weight and sum them

    sim_items['finalPrediction'] = sim_items.apply(lambda x: (x['iPrediction'] * item_weight) + (x['uPrediction'] * (1-item_weight)), axis=1)

    #get the top N users who rated this highl
    sim_items = sim_items.sort_values('finalPrediction', ascending=False)
    return sim_items.head(N)



hybrid(bx, 31315, knn, svd, .6, 10)    


,ISBN,iPrediction,uPrediction,finalPrediction
352,446606189,8.835329,7.387121,8.256046
179,440234743,8.786790,7.333763,8.205579
359,142000205,8.391583,6.847254,7.773851
297,786881852,6.321247,5.910907,6.157111
60,440214041,5.330645,5.854014,5.539993
58,312195516,3.746369,5.470936,4.436196
160,312995423,4.319208,3.501948,3.992304
35,60502258,3.518893,4.631996,3.964134
11,61009059,3.944621,3.961251,3.951273
55,380002930,3.689170,4.232122,3.906351
